In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import Adam
import time


In [47]:
# Load the dataset
df = pd.read_csv('../../dataset/preliminary_dataset.csv')  

# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# X_scaled = X

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [48]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(80000, 6) (80000,) (100000, 6)


# Model

In [49]:
def build_ann():
    model = Sequential()
    
    # Input layer
    model.add(Input(shape=(X_train.shape[1],)))
    
    # Fully connected (dense) hidden layer
    model.add(Dense(units=100, activation='relu'))
    
    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])
    
    return model

In [50]:
# Instantiate and train the model
model = build_ann()
import time
batch_size = X_train.shape[0] 
start_time=time.time()
history = model.fit(
    X_train, y_train,
    epochs=50,
    # batch_size=batch_size,
    steps_per_epoch=1,
    verbose=1
)
end_time = time.time()


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.6608 - mae: 0.4759 - mse: 0.2341
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.6524 - mae: 0.4715 - mse: 0.2300
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.6441 - mae: 0.4672 - mse: 0.2261
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.6359 - mae: 0.4629 - mse: 0.2222
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.6279 - mae: 0.4587 - mse: 0.2183
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.6200 - mae: 0.4544 - mse: 0.2145
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.6123 - mae: 0.4503 - mse: 0.2108
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.6047 - mae: 0.4461 - mse: 0.2072
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.5972 - mae: 0.4420 - mse: 0.2037
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.5898 - mae: 0.4379 - mse: 0.2002
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.5826 - mae: 0.4339 - 

In [51]:
# Calculate total training time
total_training_time = end_time - start_time  # in seconds
epochs = 50 # e.g., 50
steps_per_epoch = 1# e.g., 2000

# Total steps
total_steps = epochs * steps_per_epoch

# Calculate time per step
time_per_step = total_training_time / total_steps  # in seconds
time_per_step_ms = time_per_step * 1000  # Convert to milliseconds
#micro seconds
time_per_step_us = time_per_step_ms * 1000

# Calculate time per epoch
time_per_epoch = total_training_time / epochs

# Display results
print(f"Total Training Time: {total_training_time:.2f} sec")
print(f"Total Steps: {total_steps}")
print(f"Time per Step: {time_per_step:.6f} sec ({time_per_step_ms:.2f} ms)")
print(f"Time per Step: {time_per_step_us:.2f} µs")
print(f"Time per Epoch: {time_per_epoch:.2f} sec")


Total Training Time: 4.96 sec
Total Steps: 50
Time per Step: 0.099173 sec (99.17 ms)
Time per Step: 99172.53 µs
Time per Epoch: 0.10 sec


In [52]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Make predictions
y_pred = model.predict(X_test)

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [53]:
# # Convert continuous output to binary (0 or 1) using a threshold

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for y_pred (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')

#ACCUARCY
y_pred = (y_pred > 0.5).astype(int)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100}')


Mean Squared Error for y_pred (MSE): 0.0994728431105614
Root Mean Squared Error (RMSE): 0.3153931564104734
Accuracy: 96.96000000000001


In [54]:
# import matplotlib.pyplot as plt

# # Visualizing the loss, MSE, and MAE
# plt.figure(figsize=(15, 5))

# # Loss plot
# plt.subplot(1, 3, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MSE plot
# plt.subplot(1, 3, 2)
# plt.plot(history.history['mse'], label='Train MSE')
# plt.plot(history.history['val_mse'], label='Validation MSE')
# plt.title('Mean Squared Error')
# plt.ylabel('MSE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MAE plot
# plt.subplot(1, 3, 3)
# plt.plot(history.history['mae'], label='Train MAE')
# plt.plot(history.history['val_mae'], label='Validation MAE')
# plt.title('Mean Absolute Error')
# plt.ylabel('MAE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# plt.tight_layout()
# plt.show()

In [55]:
# import time

# # Start time
# start_time = time.time()

# # Make predictions
# y_pred = model.predict(X_test)

# # End time
# end_time = time.time()

# # Total AES time
# total_time = end_time - start_time  

# # AES per sample
# aes_per_sample = (total_time / len(X_test)) * 1e6  # Convert to microseconds (µs)

# print(f'Total AES Time: {total_time:.6f} seconds')
# print(f'AES Time per Sample: {aes_per_sample:.2f} µs/sample')